# Maryam Hosseinali - 610398209 - HW4 - IR

## Required Libraries

In [1]:

import os
import pandas as pd
import tarfile
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk


nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /home/ghadyani/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ghadyani/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Access and Load Data

In [2]:

tar_file_path = "aclImdb_v1.tar.gz"  
extracted_folder_path = "aclImdb"  


with tarfile.open(tar_file_path, "r:gz") as tar:
    tar.extractall(path=extracted_folder_path)




In [3]:
# load data from 'pos' and 'neg' folders
def load_data(folder_path):
    data = []
    labels = []
    
    for label in ['pos', 'neg']:
        folder = os.path.join(folder_path, label)
        for file_name in os.listdir(folder):
            file_path = os.path.join(folder, file_name)
            with open(file_path, 'r', encoding='utf-8') as f:
                data.append(f.read().strip())
                labels.append(1 if label == 'pos' else 0) 
                # 1 pos and 0 neg
    return pd.DataFrame({'text': data, 'label': labels})

base_folder = os.path.join(extracted_folder_path, "aclImdb")
train_folder = os.path.join(base_folder, "train")
test_folder = os.path.join(base_folder, "test")

train_data = load_data(train_folder)
test_data = load_data(test_folder)

print("Train data sample:")
print(train_data.head())
print("Test data sample:")
print(test_data.head())


Train data sample:
                                                text  label
0  I enjoyed this movie,and after watching it,it ...      1
1  The making of The Thief Of Bagdad is quite a s...      1
2  This is a very good movie. Do you want to know...      1
3  Haven't seen any of the Japanese Grudge-films,...      1
4  A young solicitor in sent to a remote area to ...      1
Test data sample:
                                                text  label
0  BROADCAST NEWS opens with a series of brief vi...      1
1  I have seen most of the Tarzan episodes. Certa...      1
2  Three kids are born during a solar eclipse and...      1
3  There is a serious scene in this movie. A scen...      1
4  I originally saw this film while I was working...      1


### load labeled text data from Dataset, organized in pos and neg folders, into structured pandas DataFrames.

# 1. Document Preprocessing

In [4]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def preprocess_text(text):
  
    text = text.lower()
    tokens = word_tokenize(text) 
    # token into words
    
    tokens = [word for word in tokens if word.isalnum()]  
    # remove punctuation
    
    tokens = [word for word in tokens if word not in stop_words] 
    # remove stopwords
    
    tokens = [stemmer.stem(word) for word in tokens] 
    #  stemming
    
    return tokens

# preprocess train and test datasets
train_data['processed_text'] = train_data['text'].apply(preprocess_text)
test_data['processed_text'] = test_data['text'].apply(preprocess_text)


print("Sample Processed Train Data:")
print(train_data[['text', 'processed_text']].head())

Sample Processed Train Data:
                                                text  \
0  I enjoyed this movie,and after watching it,it ...   
1  The making of The Thief Of Bagdad is quite a s...   
2  This is a very good movie. Do you want to know...   
3  Haven't seen any of the Japanese Grudge-films,...   
4  A young solicitor in sent to a remote area to ...   

                                      processed_text  
0  [enjoy, movi, watch, made, wonder, mani, rose,...  
1  [make, thief, bagdad, quit, stori, unto, almos...  
2  [good, movi, want, know, real, reason, mani, k...  
3  [seen, japanes, realli, enjoy, one, rare, get,...  
4  [young, solicitor, sent, remot, area, wrap, es...  


## **Document Preprocessing**

#### Prepare raw text data for ML classification by applying preprocessing techniques.

## **Steps**
1. Convert text to lowercase.
2. Tokenize reviews into individual words.
3. Remove punctuation and non-alphanumeric characters.
4. Filter out common stopwords.
5. Apply stemming to reduce words to their root forms.

## **Results**
- Processed reviews are stored in a new column processed_text.


 # 2. Naive Bayes Classification

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

# convert processed text to strings for TF-IDF
train_texts = train_data['processed_text'].apply(lambda x: ' '.join(x))
test_texts = test_data['processed_text'].apply(lambda x: ' '.join(x))

# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X_train = vectorizer.fit_transform(train_texts)
X_test = vectorizer.transform(test_texts)

# labels
y_train = train_data['label']
y_test = test_data['label']

# train Naive Bayes Classifier
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train, y_train)




MultinomialNB()

## **Naive Bayes Classification**

### Train Naive Bayes classifier to categorize reviews as positive or negative based on textual content.

## **Steps**
1. Convert preprocessed text into TF-IDF feature vectors.
2. Use top 5,000 most frequent words as features for efficiency.
3. Train a Multinomial Naive Bayes classifier on the training dataset.

## **Results**
- **TF-IDF Transformation Complete:**
  - Training Data Shape: (25,000, 5000)
  - Test Data Shape: (25,000, 5000)
  


# 3. Evaluation on Test Dataset

In [6]:



y_pred = nb_classifier.predict(X_test)
# predict test data

# eval Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Evaluation Metrics on Test Dataset:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")


print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Evaluation Metrics on Test Dataset:
Accuracy: 0.8336
Precision: 0.8492
Recall: 0.8114
F1-Score: 0.8298

Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.86      0.84     12500
           1       0.85      0.81      0.83     12500

    accuracy                           0.83     25000
   macro avg       0.83      0.83      0.83     25000
weighted avg       0.83      0.83      0.83     25000



## Evaluation on Test Dataset

#### Evaluate the trained Naive Bayes classifier on the test dataset using key classification metrics.

### Steps
1. Predict sentiments of reviews in the test dataset using the trained Naive Bayes model.
2. Calculate evaluation metrics:
   - **Accuracy**: Overall correctness of the model.
   - **Precision**: Proportion of true positive predictions among all positive predictions.
   - **Recall**: Proportion of true positives identified among all actual positives.
   - **F1-Score**: Harmonic mean of precision and recall.
3. Generate a detailed classification report for further insights.

### Results
- **Accuracy**: 0.8336  
- **Precision**: 0.8492  
- **Recall**: 0.8114  
- **F1-Score**: 0.8298  

#### **Classification Report**
| Sentiment | Precision | Recall | F1-Score | Support |
|-----------|-----------|--------|----------|---------|
| Negative  | 0.82      | 0.86   | 0.84     | 12500   |
| Positive  | 0.85      | 0.81   | 0.83     | 12500   |

### Insights
- The Naive Bayes classifier achieves **83.36% accuracy**, showing high overall performance for binary sentiment classification.
- **Precision** of 84.92% indicates strong ability to minimize false positives.
- **Recall** of 81.14% reflects good capability in capturing true positives, though slightly lower than precision.
- **F1-Score** of 82.98% balances precision and recall, confirming robust classification.
- **Classification Report** demonstrates balanced performance across negative and positive sentiment classes, with comparable precision, recall, and F1-scores for both classes.

### Key Observations
- Positive reviews show slightly higher precision, while negative reviews have better recall.
- The classifier performs consistently well across both sentiment categories, making it a reliable model for sentiment analysis tasks.


# **Overall Insights and Challenges**

## **Preprocessing Challenges**
1. **Handling Imbalanced Tokenization**:
   - Certain reviews had highly imbalanced token counts, either too short or excessively long. Preprocessing standardized these to manageable tokens.
   - For example, excessive punctuation and emoticons were removed during tokenization.

2. **Stopword Fine-Tuning**:
   - The standard NLTK stopwords set was used, but it might miss domain-specific stopwords, such as frequent neutral words in movie reviews.
   - Future iterations could enhance this by adding custom domain-specific stopwords.

3. **Stemming Trade-offs**:
   - Stemming reduces words to their root forms but can lose semantic precision. This might impact reviews with nuanced expressions.

---

## **Naive Bayes Classifier Observations**
1. **Training Data Efficiency**:
   - TF-IDF with 5,000 features ensured the Naive Bayes classifier could handle high-dimensional data efficiently without overfitting.
   - Sparse matrix representation made the large dataset computationally manageable.

2. **Word Independence Assumption**:
   - Naive Bayes assumes word independence, which, while simplifying calculations, does not model contextual dependencies between words.
   - This limitation is particularly apparent in sarcastic reviews or those with conflicting phrases (based on my own experience working with sarcastic texts).

3. **Handling Class Balance**:
   - The dataset was inherently balanced with equal positive and negative reviews, which simplified the classifier's learning process and evaluation.

---

## **Evaluation Challenges**
1. **Threshold Sensitivity**:
   - While the model achieved strong performance metrics, the binary classification threshold could be adjusted to prioritize precision or recall based on specific use cases.

2. **Generalization Beyond Dataset**:
   - Although the classifier performed well on the test dataset, its generalization to reviews with more complex vocabulary or ambiguous sentiment remains untested.


